In [1]:
import pandas as pd
import numpy as np

In [27]:

for iter, row in df.iterrows():
    if row[iter] != 0:
        print(row[iter])
    # for i in row[iter]:
    #     print(i)
new_columns = ["path_location", "Kernel", "BIC"]

new_df

[(1, 0), 'RBF+AR2+RBF', -2296.680419921875]
[(2, 1), 'RBF+AR2*Per_Year+Mat_2.5', nan]
[(3, 2), 'RBF+AR2*Per_Year*RBF+Mat_1.5', -2010.8145751953125]
[(4, 3), 'RBF+AR2*Per_Year*RBF*Mat_1.5+Mat_0.5', -1386.2291259765625]


0   0                     [(0, 0), 'RBF', -2010.927734375]
    1          [(1, 0), 'RBF+AR2+RBF', -2296.680419921875]
    2    [(2, 0), 'RBF+AR2*Per_Year+RBF', -2292.8698730...
    3    [(3, 0), 'RBF+AR2*Per_Year*RBF+RBF', -2308.547...
    4    [(4, 0), 'RBF+AR2*Per_Year*RBF*Mat_1.5+RBF', -...
                               ...                        
22  0          [(0, 22), 'RBF*Per_Month', 1889.4150390625]
    1    [(1, 22), 'RBF+AR2*Per_Month', 290.2250061035156]
    2    [(2, 22), 'RBF+AR2*Per_Year*Per_Month', -2197....
    3    [(3, 22), 'RBF+AR2*Per_Year*RBF*Per_Month', -1...
    4    [(4, 22), 'RBF+AR2*Per_Year*RBF*Mat_1.5*Per_Mo...
Length: 115, dtype: object

,path_location,Kernel,BIC
0,"(0, 0)",'RBF',-2010.927734
1,"(1, 0)",'RBF+AR2+RBF',-2296.680420
2,"(2, 0)",'RBF+AR2*Per_Year+RBF',-2292.869873
3,"(3, 0)",'RBF+AR2*Per_Year*RBF+RBF',-2308.547363
4,"(4, 0)",'RBF+AR2*Per_Year*RBF*Mat_1.5+RBF',-2326.375732
...,...,...,...
110,"(0, 22)",'RBF*Per_Month',1889.415039
111,"(1, 22)",'RBF+AR2*Per_Month',290.225006
112,"(2, 22)",'RBF+AR2*Per_Year*Per_Month',-2197.544922
113,"(3, 22)",'RBF+AR2*Per_Year*RBF*Per_Month',-1335.378174
